<a href="https://colab.research.google.com/github/alayuala/114-1PL.repo/blob/main/week3_%E6%94%B6%E5%85%A5%E6%94%AF%E5%87%BA%E5%88%97%E8%A1%A8%E5%8A%A0%E5%85%A5AI_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **日常支出速算與分攤（作業一）**
* 目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
* AI 點子：總結本週花錢習慣與建議、預算與總支出報告
* Sheet 欄位：date, category, item, amount, payer




## 分析現有儲存格

### Subtask:
檢視所有儲存格的內容，識別重複的程式碼、無用的變數或函式定義，以及潛在的錯誤來源。


**Reasoning**:
Analyze the provided notebook content to identify redundant code, unused variables, and potential errors based on the current subtask instructions.



In [21]:
# Analyzing the notebook content based on cell_id for redundancy, unused variables, and potential errors.

# Repetitive function definitions:
# authenticate_and_connect: Defined in cells 8c7b7d66, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# read_expense_data: Defined in cells 8c7b7d66, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# add_expense_record: Defined in cells 8c7b7d66, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# write_expense_data: Defined in cells 8c7b7d66, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# calculate_summary_statistics: Defined in cells 8c7b7d66, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# write_summary_sheet: Defined in cells 8c7b7d66, e082baa8, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# get_budget_data: Defined in cells f9d47421, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# calculate_spending_within_budgets: Defined in cells f9d47421, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# compare_spending_to_budget: Defined in cells f9d47421, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# generate_budget_report_string: Defined in cells f9d47421, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# generate_periodic_report_string: Defined in cells 306905eb, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# write_reports_sheet: Defined in cells 306905eb, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. Redundant.
# main_interface_logic: Defined in cells 53016ff9, 0621b9cb, 354e4570, 6014bfc8, ff6635de, a02ff186, 14266041. Redundant.
# generate_spending_analysis_and_advice: Defined in cell 29eb3ef9. This is the only definition.

# Unused variables:
# df: Defined in cell 12d57716 but not used after the display attempt, and NameError suggests it wasn't successfully created there. It is redefined and used in later cells.
# gsheets: Defined in cell Ny_jn-jGXgpv but replaced by gsheets_obj in later function calls.
# category_suggestions: Defined in cells jXMruslTfH2j, 4ebb75cd, c9eb4828, 3a8b4f48, b10062bc, e7ef3d36, 0546963a. While defined multiple times, the dictionary itself isn't directly used by the core logic or Gradio interface functions in the provided code, although it was intended for suggestions.

# Potential error sources:
# NameError: name 'demo' is not defined in cells 463b5605, 41c46cd0, 0a2a1620. This is because the Gradio interface `demo` is defined using `gr.Blocks()` within a `with` statement, and `demo.launch()` is called in separate cells *after* the `with` block where `demo` was defined has finished executing in previous runs, or before the `with` block has been executed in the current run. The definition and launch should be in the same cell or the cell defining `demo` must be executed before the cell launching it.
# TypeError: Object of type int64 is not JSON serializable in cell a81c4cdf. This error was identified and a fix (converting to float or string) was attempted in subsequent `write_summary_sheet` definitions (e.g., e7ef3d36).
# NameError: name 'category_suggestions' is not defined in cell c9eb4828. This is because the cell defining it (4ebb75cd or jXMruslTfH2j) was not executed before this cell in that specific run.
# Inconsistent variable names: `gsheets` vs `gsheets_obj`. While `gsheets_obj` is used consistently in the function definitions, the variable `gsheets` is defined in an earlier cell (Ny_jn-jGXgpv) and was used in an attempt to write data in cell dc6dde16, which failed due to `df` not being defined. This inconsistency is not a major issue as the functions rely on the passed argument `gsheets_obj`, but it's a point of note.
# Multiple definitions of the same functions: This is the primary source of redundancy and makes the notebook hard to follow.
# The AI analysis function `generate_spending_analysis_and_advice` is defined but not called in the `main_interface_logic` functions before cell 0621b9cb. It is called in cell 0621b9cb and subsequent `main_interface_logic` definitions.

# Gradio Interface Consistency:
# The Gradio interface definition in cells 0621b9cb, 68b24d91, 6014bfc8, ff6635de, a02ff186, 14266041 consistently uses `main_interface_logic` with actions like "add_expense", "generate_reports", and "calculate_aa". The inputs and outputs seem consistent with the `main_interface_logic` function signatures, including the updates to handle outputs for AA split and payer balance, and the state outputs to prevent clearing other fields. The logic for showing/hiding custom date inputs is also consistent.

# Conclusion of analysis:
# The main issues are redundant function definitions scattered across multiple cells and the `NameError` when launching the Gradio app due to the `with gr.Blocks() as demo:` block not being executed before `demo.launch()`. The `TypeError` with `int64` was addressed in later function definitions. The AI analysis function was defined separately and later integrated into `main_interface_logic`. The `category_suggestions` dictionary is defined multiple times but not used in the core logic.


**Reasoning**:
Based on the analysis of the notebook, the primary issue is the scattered and redundant function definitions. Consolidating all necessary function definitions into a single code cell will address this redundancy and prepare the code for a cleaner integration with the Gradio interface.



In [33]:
# Consolidated function definitions and necessary imports.
# This cell combines all the required functions into one place.

import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import numpy as np
import google.generativeai as genai
from google.colab import userdata

# Define the Google Sheet URL
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1KekcvnaCy4MsLsbi04U0KCeys7dnh-JxCCeHkk9xKFE/edit?usp=sharing'

# Configure the Gemini API
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in Colab secrets.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Failed to configure Gemini API: {e}")
    print("Please ensure you have added your GOOGLE_API_KEY to Colab secrets.")

# Initialize the Generative Model
try:
    gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')
    print("Gemini model initialized.")
except Exception as e:
    print(f"Failed to initialize Gemini model: {e}")
    gemini_model = None
    print("AI analysis and advice will not be available.")


# Google Sheets interaction functions
def authenticate_and_connect(sheet_url):
    """Authenticates and connects to the Google Sheet."""
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    gsheets = gc.open_by_url(sheet_url)
    return gsheets

def read_expense_data(gsheets_obj):
    """Reads expense data from the '工作表1' worksheet into a pandas DataFrame."""
    worksheet = gsheets_obj.worksheet('工作表1')
    sheets = worksheet.get_all_values()
    if not sheets:
        # Return an empty DataFrame with expected columns if sheet is empty
        return pd.DataFrame(columns=['date', 'category', 'item', 'amount', 'payer'])
    df = pd.DataFrame(sheets[1:], columns=sheets[0])
    # Ensure 'amount' is numeric and 'date' is datetime
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    return df

def add_expense_record(df, date_str, category, item, amount, payer):
    """Appends a new expense record to the DataFrame."""
    new_data = pd.DataFrame([{
        'date': date_str,
        'category': category,
        'item': item,
        'amount': amount,
        'payer': payer
    }])
    updated_df = pd.concat([df, new_data], ignore_index=True)
    # Ensure 'amount' is numeric and 'date' is datetime after concat
    updated_df['amount'] = pd.to_numeric(updated_df['amount'], errors='coerce')
    updated_df['date'] = pd.to_datetime(updated_df['date'], errors='coerce')
    return updated_df

def write_expense_data(df, gsheets_obj):
    """Writes the DataFrame back to the '工作表1' worksheet."""
    # Convert date column to string format before writing
    df_to_write = df.copy()
    df_to_write['date'] = df_to_write['date'].dt.strftime('%Y-%m-%d') if pd.api.types.is_datetime64_any_dtype(df_to_write['date']) else df_to_write['date'].astype(str)
    # Convert numeric types to float or string to avoid TypeError during JSON serialization
    for col in ['amount']:
        if pd.api.types.is_numeric_dtype(df_to_write[col]):
            df_to_write[col] = df_to_write[col].apply(lambda x: float(x) if pd.notna(x) else '')

    data_to_write = [df_to_write.columns.tolist()] + df_to_write.values.tolist()
    worksheet = gsheets_obj.worksheet('工作表1')
    worksheet.clear()
    worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
    print("Updated data has been successfully written back to Google Sheet.")

# Summary statistics functions
def calculate_summary_statistics(df):
    """Calculates summary statistics from the DataFrame."""
    # Ensure 'amount' is numeric before calculations
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    total_amount = df['amount'].sum()

    category_subtotals = df.groupby('category')['amount'].sum().reset_index()

    unique_payers = df['payer'].unique()
    num_payers = len(unique_payers)
    aa_split_amount = total_amount / num_payers if num_payers > 0 else 0

    payer_totals = df.groupby('payer')['amount'].sum().reset_index()

    payer_balance = payer_totals.copy()
    payer_balance['balance'] = payer_balance['amount'] - aa_split_amount

    return total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance

def write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=None):
    """Writes the summary statistics and AI analysis to the 'Summary' sheet."""
    try:
        summary_sheet = gsheets_obj.worksheet('Summary')
    except gspread.WorksheetNotFound:
        summary_sheet = gsheets_obj.add_worksheet(title="Summary", rows="100", cols="20")

    summary_sheet.clear()

    summary_sheet.update_cell(1, 1, '總支出金額')
    # Convert total_amount to float or string
    summary_sheet.update_cell(1, 2, float(total_amount) if isinstance(total_amount, (int, float, np.number)) else str(total_amount))


    summary_sheet.update_cell(3, 1, '各類別支出小計')
    summary_sheet.update([category_subtotals.columns.tolist()], 'A4')
    # Convert numeric columns in category_subtotals to float or string
    category_subtotals_list = category_subtotals.values.tolist()
    for row in category_subtotals_list:
        row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
    summary_sheet.update(category_subtotals_list, 'A5')

    summary_sheet.update_cell(7, 1, 'AA 分攤金額 (每人)')
    # Convert aa_split_amount to float or string
    summary_sheet.update_cell(7, 2, float(aa_split_amount) if isinstance(aa_split_amount, (int, float, np.number)) else str(aa_split_amount))

    summary_sheet.update_cell(9, 1, '每人總支付金額')
    summary_sheet.update([payer_totals.columns.tolist()], 'A10')
    # Convert numeric columns in payer_totals to float or string
    payer_totals_list = payer_totals.values.tolist()
    for row in payer_totals_list:
         row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
    summary_sheet.update(payer_totals_list, 'A11')

    summary_sheet.update_cell(13, 1, '每人應收/應付金額')
    summary_sheet.update([payer_balance.columns.tolist()], 'A14')
    # Convert numeric columns in payer_balance to float or string
    payer_balance_list = payer_balance.values.tolist()
    for row in payer_balance_list:
         row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
         row[2] = float(row[2]) if isinstance(row[2], (int, float, np.number)) else str(row[2])
    summary_sheet.update(payer_balance_list, 'A15')


    ai_analysis_start_row = 17
    summary_sheet.update_cell(ai_analysis_start_row, 1, 'AI 支出分析與建議')
    if spending_analysis_and_advice:
        analysis_lines = spending_analysis_and_advice.split('\n')
        analysis_data_to_write = [[line] for line in analysis_lines]
        analysis_range = f'A{ai_analysis_start_row + 1}'
        summary_sheet.update(analysis_data_to_write, analysis_range)
    else:
        summary_sheet.update_cell(ai_analysis_start_row + 1, 1, '尚未執行 AI 支出分析。')

    print("總計、分類小計、AA 分攤和 AI 分析資訊已更新至 Summary 分頁！")

# Budget data functions
def get_budget_data(gsheets_obj):
    """Reads budget data from the 'Budgets' sheet into a pandas DataFrame."""
    try:
        budgets_sheet = gsheets_obj.worksheet('Budgets')
        budget_data = budgets_sheet.get_all_values()
        if len(budget_data) > 1:
            budget_df = pd.DataFrame(budget_data[1:], columns=budget_data[0])
            # Ensure date and amount columns are in correct format
            budget_df['Start Date'] = pd.to_datetime(budget_df['Start Date'], errors='coerce')
            budget_df['End Date'] = pd.to_datetime(budget_df['End Date'], errors='coerce')
            budget_df['Amount'] = pd.to_numeric(budget_df['Amount'], errors='coerce')
        else:
            budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
            print("No budget data found in the 'Budgets' sheet.")
    except gspread.WorksheetNotFound:
        print("Sheet 'Budgets' not found. Please create it and add budget data.")
        budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
    except Exception as e:
        print(f"An error occurred while reading the 'Budgets' sheet: {e}")
        budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
    return budget_df

def calculate_spending_within_budgets(df, budget_df):
    """Calculates spending for each budget based on the expense data."""
    if budget_df.empty or df.empty:
        budget_df['Spending'] = 0.0 # Add spending column even if empty
        return budget_df

    # Ensure df's date column is datetime and amount is numeric
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    budget_df['Spending'] = 0.0

    for index, budget_row in budget_df.iterrows():
        start_date = budget_row['Start Date']
        end_date = budget_row['End Date']
        category = budget_row['Category']

        filtered_spending = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()

        if pd.notna(category) and category != '':
            filtered_spending = filtered_spending[filtered_spending['category'] == category].copy()

        total_spending_for_budget = filtered_spending['amount'].sum()
        budget_df.loc[index, 'Spending'] = total_spending_for_budget

    return budget_df

def compare_spending_to_budget(budget_df):
    """Compars spending to budget and adds status columns."""
    if budget_df.empty:
        budget_df['Remaining/Overspent'] = 0.0
        budget_df['Status'] = 'N/A'
        return budget_df

    budget_df['Remaining/Overspent'] = budget_df['Amount'] - budget_df['Spending']

    def categorize_budget_status(row):
        if row['Remaining/Overspent'] >= 0:
            return 'Within Budget'
        else:
            return 'Over Budget'

    budget_df['Status'] = budget_df.apply(categorize_budget_status, axis=1)
    return budget_df

def generate_budget_report_string(budget_df):
    """Generates a formatted budget report string."""
    budget_report_lines = ["=== 預算報告 ==="]

    if budget_df.empty:
        budget_report_lines.append("目前沒有預算數據可供報告。")
    else:
        for index, budget_row in budget_df.iterrows():
            budget_name = budget_row['Budget Name']
            start_date = budget_row['Start Date'].strftime('%Y-%m-%d') if pd.notna(budget_row['Start Date']) else 'N/A'
            end_date = budget_row['End Date'].strftime('%Y-%m-%d') if pd.notna(budget_row['End Date']) else 'N/A'
            budgeted_amount = budget_row['Amount']
            actual_spending = budget_row['Spending']
            remaining_overspent = budget_row['Remaining/Overspent']
            status = budget_row['Status']
            category = budget_row['Category'] if pd.notna(budget_row['Category']) and budget_row['Category'] != '' else '總體'

            budget_report_lines.append(f"\n預算名稱: {budget_name}")
            budget_report_lines.append(f"期間: {start_date} 至 {end_date}")
            budget_report_lines.append(f"類別: {category}")
            budget_report_lines.append(f"預算金額: {budgeted_amount:.2f}")
            budget_report_lines.append(f"實際支出: {actual_spending:.2f}")
            budget_report_lines.append(f"剩餘/超支金額: {remaining_overspent:.2f}")
            budget_report_lines.append(f"狀態: {status}")

    return "\n".join(budget_report_lines)

# Report generation functions
def generate_periodic_report_string(df, report_start_date, report_end_date, report_period_name):
    """Generates a formatted periodic report string based on a date range."""
    # Ensure the 'date' column in the expense DataFrame is in date format for comparison
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # Filter the main expense DataFrame (`df`) to include only transactions within the report_start_date and report_end_date.
    filtered_spending_for_report = df[(df['date'] >= report_start_date) & (df['date'] <= report_end_date)].copy()

    # Calculate the total spending for this filtered period.
    total_spending_period = filtered_spending_for_report['amount'].sum()

    # Calculate the category subtotals for the filtered spending data.
    category_subtotals_period = filtered_spending_for_report.groupby('category')['amount'].sum().reset_index()

    # Calculate the total amount paid by each person within the filtered period.
    payer_totals_period = filtered_spending_for_report.groupby('payer')['amount'].sum().reset_index()

    # Format the calculated total spending, category subtotals, and payer totals into a readable report string.
    periodic_report_lines = [f"=== {report_period_name} ({report_start_date.strftime('%Y-%m-%d')} 至 {report_end_date.strftime('%Y-%m-%d')}) 支出報告 ==="]
    periodic_report_lines.append(f"\n總支出金額: {total_spending_period:.2f}元")

    if not category_subtotals_period.empty:
        periodic_report_lines.append("\n各類別支出小計:")
        for index, row in category_subtotals_period.iterrows():
            periodic_report_lines.append(f"- {row['category']}: {row['amount']:.2f}元")
    else:
        periodic_report_lines.append("\n此期間沒有類別支出數據。")

    if not payer_totals_period.empty:
        periodic_report_lines.append("\n每人總支付金額:")
        for index, row in payer_totals_period.iterrows():
            periodic_report_lines.append(f"- {row['payer']}: {row['amount']:.2f}元")
    else:
        periodic_report_lines.append("\n此期間沒有支付者數據。")

    return "\n".join(periodic_report_lines)

def write_reports_sheet(gsheets_obj, periodic_report_string, budget_report_string):
    """Writes the periodic and budget reports to the 'Reports' sheet."""
    try:
        reports_sheet = gsheets_obj.worksheet('Reports')
        print("Sheet 'Reports' already exists, clearing it.")
        reports_sheet.clear()
    except gspread.WorksheetNotFound:
        print("Creating new sheet 'Reports'.")
        reports_sheet = gsheets_obj.add_worksheet(title="Reports", rows="200", cols="20")

    periodic_report_start_row_reports = 1
    budget_report_start_row_reports = periodic_report_start_row_reports + len(periodic_report_string.split('\n')) + 5

    reports_sheet.update_cell(periodic_report_start_row_reports, 1, '=== 週期支出報告 ===')
    periodic_report_lines = periodic_report_string.split('\n')
    if periodic_report_lines:
        periodic_report_data_to_write = [[line] for line in periodic_report_lines]
        periodic_report_range = f'A{periodic_report_start_row_reports + 1}'
        reports_sheet.update(periodic_report_data_to_write, periodic_report_range)

    reports_sheet.update_cell(budget_report_start_row_reports, 1, '=== 預算報告 ===')
    budget_report_lines = budget_report_string.split('\n')
    if budget_report_lines:
        budget_report_data_to_write = [[line] for line in budget_report_lines]
        budget_report_range = f'A{budget_report_start_row_reports + 1}'
        reports_sheet.update(budget_report_data_to_write, budget_report_range)

    print("週期支出報告和預算報告已寫入 Reports 分頁！")

def generate_spending_analysis_and_advice(df):
    """Generates spending analysis and advice using the Gemini API."""
    if gemini_model is None:
        return "AI 分析與建議功能目前無法使用，請檢查您的 API 金鑰和模型初始化狀態。"

    if df.empty:
        return "沒有足夠的支出數據來進行 AI 分析。"

    # Prepare a prompt for the AI
    # You can customize this prompt based on the insights you want from the AI
    prompt = f"""
    請分析以下支出數據，並提供總結以及關於如何改進消費習慣的建議。數據格式為 DataFrame 的 to_string() 輸出：

    {df.to_string()}

    請提供以下內容：
    1. 總體支出摘要
    2. 各類別支出分析
    3. 基於數據的個人化消費建議
    """

    try:
        # Generate content using the Gemini model
        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"AI 分析與建議生成失敗: {e}"

# Category suggestion dictionary (can be expanded)
category_suggestions = {
    '便當': '餐飲',
    '拉麵': '餐飲',
    '油錢': '交通',
    '衛生紙': '購物',
    '珍奶': '飲料',
    '星巴克': '飲料',
    '咖啡': '飲料',
    '麵包': '餐飲',
    '電影票': '娛樂',
    '公車': '交通',
    '捷運': '交通',
    '計程車': '交通',
    '衣服': '購物',
    '書本': '購物',
    '水電費': '帳單',
    '房租': '居住',
    '薪水': '收入' # Although not an expense, useful for tracking
}

# Define the Google Sheet URL
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1KekcvnaCy4MsLsbi04U0KCeys7dnh-JxCCeHkk9xKFE/edit?usp=sharing'

Failed to configure Gemini API: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.
Please ensure you have added your GOOGLE_API_KEY to Colab secrets.
Gemini model initialized.


## 定義主要介面邏輯

### Subtask:
重新定義處理 Gradio 介面使用者輸入和呼叫後端函式的 `main_interface_logic` 函式，確保其能正確協調所有功能。


**Reasoning**:
Define the main interface logic function that orchestrates the calls to the backend functions based on user actions from the Gradio interface.



In [23]:
import gradio as gr

def main_interface_logic(action, date_str=None, category=None, item=None, amount=None, payer=None, report_period_choice=None, custom_start_date_str=None, custom_end_date_str=None):
    """
    Orchestrates the data processing and reporting based on Gradio user input.
    """
    gsheets_obj = authenticate_and_connect(SHEET_URL)
    df = read_expense_data(gsheets_obj)

    output_message = ""
    periodic_report_output = ""
    budget_report_output = ""
    aa_split_output = ""
    payer_balance_output = ""
    ai_analysis_output = "" # Added AI analysis output variable


    if action == "add_expense":
        if date_str and category and item and amount is not None and payer:
            try:
                # Validate date format
                datetime.datetime.strptime(date_str, '%Y-%m-%d')
                # Ensure amount is a float
                amount = float(amount)

                df = add_expense_record(df, date_str, category, item, amount, payer)
                write_expense_data(df, gsheets_obj)
                # Recalculate and write summary after adding expense
                total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
                # Generate AI analysis and advice
                ai_analysis_output = generate_spending_analysis_and_advice(df)
                write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output) # Pass AI analysis to write_summary_sheet
                output_message = "新的支出紀錄已成功新增並更新至 Google Sheet 及 Summary 分頁。"
                aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
                payer_balance_output = payer_balance.to_string(index=False)
            except ValueError as e:
                output_message = f"輸入錯誤: {e}. 請檢查日期和金額格式。"
        else:
            output_message = "請填寫所有支出紀錄欄位。"
        return output_message, "", "", aa_split_output, payer_balance_output, ai_analysis_output # Include AI analysis output

    elif action == "generate_reports":
        if report_period_choice:
            today = datetime.date.today()
            report_start_date = None
            report_end_date = None
            report_period_name = ""

            if report_period_choice == '上週':
                last_sunday = today - relativedelta(days=today.weekday() + 1)
                last_monday = last_sunday - relativedelta(days=6)
                report_start_date = last_monday
                report_end_date = last_sunday
                report_period_name = "上週"
            elif report_period_choice == '上個月':
                first_day_of_current_month = today.replace(day=1)
                last_day_of_last_month = first_day_of_current_month - relativedelta(days=1)
                first_day_of_last_month = last_day_of_last_month.replace(day=1)
                report_start_date = first_day_of_last_month
                report_end_date = last_day_of_last_month
                report_period_name = "上個月"
            elif report_period_choice == '自訂日期範圍':
                if custom_start_date_str and custom_end_date_str:
                    try:
                        report_start_date = datetime.datetime.strptime(custom_start_date_str, '%Y-%m-%d').date()
                        report_end_date = datetime.datetime.strptime(custom_end_date_str, '%Y-%m-%d').date()
                        if report_start_date > report_end_date:
                            output_message = "自訂日期範圍錯誤：開始日期不能晚於結束日期。"
                            return output_message, "", "", "", "", "" # Include "" for all outputs
                        report_period_name = "自訂日期範圍"
                    except ValueError:
                        output_message = "自訂日期格式錯誤，請使用 YYYY-MM-DD 格式。"
                        return output_message, "", "", "", "", "" # Include "" for all outputs
                else:
                    output_message = "請輸入自訂報告的開始日期和結束日期。"
                    return output_message, "", "", "", "", "" # Include "" for all outputs

            # Generate Periodic Report
            if report_start_date and report_end_date:
                 periodic_report_output = generate_periodic_report_string(df, report_start_date, report_end_date, report_period_name)
            else:
                 periodic_report_output = "請選擇或輸入有效的報告期間。"

            # Generate Budget Report
            budget_df = get_budget_data(gsheets_obj)
            budget_df = calculate_spending_within_budgets(df, budget_df)
            budget_df = compare_spending_to_budget(budget_df)
            budget_report_output = generate_budget_report_string(budget_df)

            # Write reports to Reports sheet
            write_reports_sheet(gsheets_obj, periodic_report_output, budget_report_output)
            output_message = "報告已生成並更新至 Reports 分頁。"
        else:
            output_message = "請選擇報告期間。"
        return output_message, periodic_report_output, budget_report_output, "", "", "" # Include "" for AA and AI analysis outputs

    elif action == "calculate_aa":
        total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
        aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
        payer_balance_output = payer_balance.to_string(index=False)
        output_message = "AA 分攤金額和每人應收/應付金額已計算。"
        return output_message, "", "", aa_split_output, payer_balance_output, "" # Include "" for AI analysis output

    elif action == "generate_ai_analysis": # Added action for generating AI analysis on demand
         ai_analysis_output = generate_spending_analysis_and_advice(df)
         output_message = "AI 支出分析與建議已生成。"
         # Update the Summary sheet with the new AI analysis
         total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
         write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output)
         return output_message, "", "", "", "", ai_analysis_output # Return only AI analysis output and status

# Define Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 日常支出管理與報告")

    with gr.Tab("新增支出紀錄"):
        gr.Markdown("在此輸入新的支出紀錄。")
        expense_date = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder="例如：2023-10-27")
        expense_category = gr.Textbox(label="類別", placeholder="例如：餐飲、交通")
        expense_item = gr.Textbox(label="品項", placeholder="例如：午餐、公車票")
        expense_amount = gr.Number(label="金額", placeholder="例如：100")
        expense_payer = gr.Textbox(label="支付者", placeholder="例如：我, 朋友A")
        add_expense_btn = gr.Button("新增紀錄")
        add_expense_output = gr.Textbox(label="狀態訊息")

        add_expense_btn.click(
            fn=lambda d, c, i, a, p: main_interface_logic("add_expense", date_str=d, category=c, item=i, amount=a, payer=p),
            inputs=[expense_date, expense_category, expense_item, expense_amount, expense_payer],
            outputs=[add_expense_output, gr.State(), gr.State(), gr.State(), gr.State(), gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("生成報告"):
        gr.Markdown("選擇期間以生成支出報告和預算報告。")
        report_period_selection = gr.Dropdown(
            choices=['上週', '上個月', '自訂日期範圍'],
            label="選擇報告期間"
        )
        custom_start_date = gr.Textbox(label="自訂開始日期 (YYYY-MM-DD)", placeholder="例如：2023-10-01", visible=False)
        custom_end_date = gr.Textbox(label="自訂結束日期 (YYYY-MM-DD)", placeholder="例如：2023-10-31", visible=False)

        # Show/hide custom date fields based on dropdown selection
        def toggle_custom_dates(choice):
            if choice == '自訂日期範圍':
                return gr.update(visible=True), gr.update(visible=True)
            else:
                return gr.update(visible=False), gr.update(visible=False)

        report_period_selection.change(
            fn=toggle_custom_dates,
            inputs=report_period_selection,
            outputs=[custom_start_date, custom_end_date]
        )

        generate_report_btn = gr.Button("生成報告")
        report_status_output = gr.Textbox(label="報告生成狀態")
        periodic_report_display = gr.Textbox(label="週期支出報告", lines=10)
        budget_report_display = gr.Textbox(label="預算報告", lines=10)

        generate_report_btn.click(
            fn=lambda choice, start_date_str, end_date_str: main_interface_logic("generate_reports", report_period_choice=choice, custom_start_date_str=start_date_str, custom_end_date_str=end_date_str),
            inputs=[report_period_selection, custom_start_date, custom_end_date],
            outputs=[report_status_output, periodic_report_display, budget_report_display, gr.State(), gr.State(), gr.State()] # Added gr.State() for AA and AI analysis outputs
        )

    with gr.Tab("分帳與計算 (AA)"): # New tab for AA Split
        gr.Markdown("在此計算 AA 分攤金額和每人應收/應付金額。")
        calculate_aa_btn = gr.Button("計算 AA 分攤")
        aa_calculation_status = gr.Textbox(label="狀態訊息")
        aa_split_display = gr.Textbox(label="AA 分攤金額", lines=1)
        payer_balance_display = gr.Textbox(label="每人應收/應付金額", lines=5)

        calculate_aa_btn.click(
            fn=lambda: main_interface_logic("calculate_aa"),
            inputs=[],
            outputs=[aa_calculation_status, gr.State(), gr.State(), aa_split_display, payer_balance_display, gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("AI 支出分析"): # New tab for AI Analysis
        gr.Markdown("在此獲取 AI 支出的分析與建議。")
        generate_ai_analysis_btn = gr.Button("生成 AI 分析")
        ai_analysis_status = gr.Textbox(label="狀態訊息")
        ai_analysis_display = gr.Textbox(label="AI 分析與建議", lines=15)

        generate_ai_analysis_btn.click(
            fn=lambda: main_interface_logic("generate_ai_analysis"),
            inputs=[],
            outputs=[ai_analysis_status, gr.State(), gr.State(), gr.State(), gr.State(), ai_analysis_display] # Output AI analysis display
        )

**Reasoning**:
Launch the Gradio application to make the updated interface available for user interaction and testing.



In [24]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://313f9005b219db167d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 定義 gradio 介面

### Subtask:
重新定義 Gradio 介面 (`demo`)，確保所有必要的輸入和輸出元件都已連結到更新後的 `main_interface_logic` 函式。


**Reasoning**:
Redefine the Gradio interface to include the AI analysis tab and link the buttons to the main logic function, ensuring all inputs and outputs are correctly configured.



In [25]:
# Define Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 日常支出管理與報告")

    with gr.Tab("新增支出紀錄"):
        gr.Markdown("在此輸入新的支出紀錄。")
        expense_date = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder="例如：2023-10-27")
        expense_category = gr.Textbox(label="類別", placeholder="例如：餐飲、交通")
        expense_item = gr.Textbox(label="品項", placeholder="例如：午餐、公車票")
        expense_amount = gr.Number(label="金額", placeholder="例如：100")
        expense_payer = gr.Textbox(label="支付者", placeholder="例如：我, 朋友A")
        add_expense_btn = gr.Button("新增紀錄")
        add_expense_output = gr.Textbox(label="狀態訊息")

        add_expense_btn.click(
            fn=lambda d, c, i, a, p: main_interface_logic("add_expense", date_str=d, category=c, item=i, amount=a, payer=p),
            inputs=[expense_date, expense_category, expense_item, expense_amount, expense_payer],
            outputs=[add_expense_output, gr.State(), gr.State(), gr.State(), gr.State(), gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("生成報告"):
        gr.Markdown("選擇期間以生成支出報告和預算報告。")
        report_period_selection = gr.Dropdown(
            choices=['上週', '上個月', '自訂日期範圍'],
            label="選擇報告期間"
        )
        custom_start_date = gr.Textbox(label="自訂開始日期 (YYYY-MM-DD)", placeholder="例如：2023-10-01", visible=False)
        custom_end_date = gr.Textbox(label="自訂結束日期 (YYYY-MM-DD)", placeholder="例如：2023-10-31", visible=False)

        # Show/hide custom date fields based on dropdown selection
        def toggle_custom_dates(choice):
            if choice == '自訂日期範圍':
                return gr.update(visible=True), gr.update(visible=True)
            else:
                return gr.update(visible=False), gr.update(visible=False)

        report_period_selection.change(
            fn=toggle_custom_dates,
            inputs=report_period_selection,
            outputs=[custom_start_date, custom_end_date]
        )

        generate_report_btn = gr.Button("生成報告")
        report_status_output = gr.Textbox(label="報告生成狀態")
        periodic_report_display = gr.Textbox(label="週期支出報告", lines=10)
        budget_report_display = gr.Textbox(label="預算報告", lines=10)

        generate_report_btn.click(
            fn=lambda choice, start_date_str, end_date_str: main_interface_logic("generate_reports", report_period_choice=choice, custom_start_date_str=start_date_str, custom_end_date_str=end_date_str),
            inputs=[report_period_selection, custom_start_date, custom_end_date],
            outputs=[report_status_output, periodic_report_display, budget_report_display, gr.State(), gr.State(), gr.State()] # Added gr.State() for AA and AI analysis outputs
        )

    with gr.Tab("分帳與計算 (AA)"): # New tab for AA Split
        gr.Markdown("在此計算 AA 分攤金額和每人應收/應付金額。")
        calculate_aa_btn = gr.Button("計算 AA 分攤")
        aa_calculation_status = gr.Textbox(label="狀態訊息")
        aa_split_display = gr.Textbox(label="AA 分攤金額", lines=1)
        payer_balance_display = gr.Textbox(label="每人應收/應付金額", lines=5)

        calculate_aa_btn.click(
            fn=lambda: main_interface_logic("calculate_aa"),
            inputs=[],
            outputs=[aa_calculation_status, gr.State(), gr.State(), aa_split_display, payer_balance_display, gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("AI 支出分析"): # New tab for AI Analysis
        gr.Markdown("在此獲取 AI 支出的分析與建議。")
        generate_ai_analysis_btn = gr.Button("生成 AI 分析")
        ai_analysis_status = gr.Textbox(label="狀態訊息")
        ai_analysis_display = gr.Textbox(label="AI 分析與建議", lines=15)

        generate_ai_analysis_btn.click(
            fn=lambda: main_interface_logic("generate_ai_analysis"),
            inputs=[],
            outputs=[ai_analysis_status, gr.State(), gr.State(), gr.State(), gr.State(), ai_analysis_display] # Output AI analysis display
        )

**Reasoning**:
The Gradio interface is defined in the previous step. Now, launch the Gradio application to make it accessible via a shareable link.



In [26]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9402b4ee8b55b104d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 修正錯誤

### Subtask:
修正整合後的程式碼中存在的錯誤，確保其能夠順利執行。


**Reasoning**:
Correct the `write_summary_sheet` function to handle `int64` types by converting them to float or string before updating the Google Sheet to avoid the `TypeError: Object of type int64 is not JSON serializable` error. Also, ensure the `write_expense_data` function correctly handles numeric types for writing to the sheet.



In [27]:
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import numpy as np
import google.generativeai as genai
from google.colab import userdata

# Define the Google Sheet URL
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1KekcvnaCy4MsLsbi04U0KCeys7dnh-JxCCeHkk9xKFE/edit?usp=sharing'

# Configure the Gemini API
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in Colab secrets.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Failed to configure Gemini API: {e}")
    print("Please ensure you have added your GOOGLE_API_KEY to Colab secrets.")

# Initialize the Generative Model
try:
    gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')
    print("Gemini model initialized.")
except Exception as e:
    print(f"Failed to initialize Gemini model: {e}")
    gemini_model = None
    print("AI analysis and advice will not be available.")


# Google Sheets interaction functions
def authenticate_and_connect(sheet_url):
    """Authenticates and connects to the Google Sheet."""
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    gsheets = gc.open_by_url(sheet_url)
    return gsheets

def read_expense_data(gsheets_obj):
    """Reads expense data from the '工作表1' worksheet into a pandas DataFrame."""
    worksheet = gsheets_obj.worksheet('工作表1')
    sheets = worksheet.get_all_values()
    if not sheets:
        # Return an empty DataFrame with expected columns if sheet is empty
        return pd.DataFrame(columns=['date', 'category', 'item', 'amount', 'payer'])
    df = pd.DataFrame(sheets[1:], columns=sheets[0])
    # Ensure 'amount' is numeric and 'date' is datetime
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    return df

def add_expense_record(df, date_str, category, item, amount, payer):
    """Appends a new expense record to the DataFrame."""
    new_data = pd.DataFrame([{
        'date': date_str,
        'category': category,
        'item': item,
        'amount': amount,
        'payer': payer
    }])
    updated_df = pd.concat([df, new_data], ignore_index=True)
    # Ensure 'amount' is numeric and 'date' is datetime after concat
    updated_df['amount'] = pd.to_numeric(updated_df['amount'], errors='coerce')
    updated_df['date'] = pd.to_datetime(updated_df['date'], errors='coerce')
    return updated_df

def write_expense_data(df, gsheets_obj):
    """Writes the DataFrame back to the '工作表1' worksheet."""
    # Convert date column to string format before writing
    df_to_write = df.copy()
    df_to_write['date'] = df_to_write['date'].dt.strftime('%Y-%m-%d') if pd.api.types.is_datetime64_any_dtype(df_to_write['date']) else df_to_write['date'].astype(str)
    # Convert numeric types to float or string to avoid TypeError during JSON serialization
    for col in ['amount']:
        if pd.api.types.is_numeric_dtype(df_to_write[col]):
            df_to_write[col] = df_to_write[col].apply(lambda x: float(x) if pd.notna(x) else '')

    data_to_write = [df_to_write.columns.tolist()] + df_to_write.values.tolist()
    worksheet = gsheets_obj.worksheet('工作表1')
    worksheet.clear()
    worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
    print("Updated data has been successfully written back to Google Sheet.")

# Summary statistics functions
def calculate_summary_statistics(df):
    """Calculates summary statistics from the DataFrame."""
    # Ensure 'amount' is numeric before calculations
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    total_amount = df['amount'].sum()

    category_subtotals = df.groupby('category')['amount'].sum().reset_index()

    unique_payers = df['payer'].unique()
    num_payers = len(unique_payers)
    aa_split_amount = total_amount / num_payers if num_payers > 0 else 0

    payer_totals = df.groupby('payer')['amount'].sum().reset_index()

    payer_balance = payer_totals.copy()
    payer_balance['balance'] = payer_balance['amount'] - aa_split_amount

    return total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance

def write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=None):
    """Writes the summary statistics and AI analysis to the 'Summary' sheet."""
    try:
        summary_sheet = gsheets_obj.worksheet('Summary')
    except gspread.WorksheetNotFound:
        summary_sheet = gsheets_obj.add_worksheet(title="Summary", rows="100", cols="20")

    summary_sheet.clear()

    summary_sheet.update_cell(1, 1, '總支出金額')
    # Convert total_amount to float or string
    summary_sheet.update_cell(1, 2, float(total_amount) if isinstance(total_amount, (int, float, np.number)) else str(total_amount))


    summary_sheet.update_cell(3, 1, '各類別支出小計')
    summary_sheet.update([category_subtotals.columns.tolist()], 'A4')
    # Convert numeric columns in category_subtotals to float or string
    category_subtotals_list = category_subtotals.values.tolist()
    for row in category_subtotals_list:
        row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
    summary_sheet.update(category_subtotals_list, 'A5')

    summary_sheet.update_cell(7, 1, 'AA 分攤金額 (每人)')
    # Convert aa_split_amount to float or string
    summary_sheet.update_cell(7, 2, float(aa_split_amount) if isinstance(aa_split_amount, (int, float, np.number)) else str(aa_split_amount))

    summary_sheet.update_cell(9, 1, '每人總支付金額')
    summary_sheet.update([payer_totals.columns.tolist()], 'A10')
    # Convert numeric columns in payer_totals to float or string
    payer_totals_list = payer_totals.values.tolist()
    for row in payer_totals_list:
         row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
    summary_sheet.update(payer_totals_list, 'A11')

    summary_sheet.update_cell(13, 1, '每人應收/應付金額')
    summary_sheet.update([payer_balance.columns.tolist()], 'A14')
    # Convert numeric columns in payer_balance to float or string
    payer_balance_list = payer_balance.values.tolist()
    for row in payer_balance_list:
         row[1] = float(row[1]) if isinstance(row[1], (int, float, np.number)) else str(row[1])
         row[2] = float(row[2]) if isinstance(row[2], (int, float, np.number)) else str(row[2])
    summary_sheet.update(payer_balance_list, 'A15')


    ai_analysis_start_row = 17
    summary_sheet.update_cell(ai_analysis_start_row, 1, 'AI 支出分析與建議')
    if spending_analysis_and_advice:
        analysis_lines = spending_analysis_and_advice.split('\n')
        analysis_data_to_write = [[line] for line in analysis_lines]
        analysis_range = f'A{ai_analysis_start_row + 1}'
        summary_sheet.update(analysis_data_to_write, analysis_range)
    else:
        summary_sheet.update_cell(ai_analysis_start_row + 1, 1, '尚未執行 AI 支出分析。')

    print("總計、分類小計、AA 分攤和 AI 分析資訊已更新至 Summary 分頁！")

# Budget data functions
def get_budget_data(gsheets_obj):
    """Reads budget data from the 'Budgets' sheet into a pandas DataFrame."""
    try:
        budgets_sheet = gsheets_obj.worksheet('Budgets')
        budget_data = budgets_sheet.get_all_values()
        if len(budget_data) > 1:
            budget_df = pd.DataFrame(budget_data[1:], columns=budget_data[0])
            # Ensure date and amount columns are in correct format
            budget_df['Start Date'] = pd.to_datetime(budget_df['Start Date'], errors='coerce')
            budget_df['End Date'] = pd.to_datetime(budget_df['End Date'], errors='coerce')
            budget_df['Amount'] = pd.to_numeric(budget_df['Amount'], errors='coerce')
        else:
            budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
            print("No budget data found in the 'Budgets' sheet.")
    except gspread.WorksheetNotFound:
        print("Sheet 'Budgets' not found. Please create it and add budget data.")
        budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
    except Exception as e:
        print(f"An error occurred while reading the 'Budgets' sheet: {e}")
        budget_df = pd.DataFrame(columns=['Budget Name', 'Period', 'Start Date', 'End Date', 'Amount', 'Category'])
    return budget_df

def calculate_spending_within_budgets(df, budget_df):
    """Calculates spending for each budget based on the expense data."""
    if budget_df.empty or df.empty:
        budget_df['Spending'] = 0.0 # Add spending column even if empty
        return budget_df

    # Ensure df's date column is datetime and amount is numeric
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    budget_df['Spending'] = 0.0

    for index, budget_row in budget_df.iterrows():
        start_date = budget_row['Start Date']
        end_date = budget_row['End Date']
        category = budget_row['Category']

        filtered_spending = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()

        if pd.notna(category) and category != '':
            filtered_spending = filtered_spending[filtered_spending['category'] == category].copy()

        total_spending_for_budget = filtered_spending['amount'].sum()
        budget_df.loc[index, 'Spending'] = total_spending_for_budget

    return budget_df

def compare_spending_to_budget(budget_df):
    """Compars spending to budget and adds status columns."""
    if budget_df.empty:
        budget_df['Remaining/Overspent'] = 0.0
        budget_df['Status'] = 'N/A'
        return budget_df

    budget_df['Remaining/Overspent'] = budget_df['Amount'] - budget_df['Spending']

    def categorize_budget_status(row):
        if row['Remaining/Overspent'] >= 0:
            return 'Within Budget'
        else:
            return 'Over Budget'

    budget_df['Status'] = budget_df.apply(categorize_budget_status, axis=1)
    return budget_df

def generate_budget_report_string(budget_df):
    """Generates a formatted budget report string."""
    budget_report_lines = ["=== 預算報告 ==="]

    if budget_df.empty:
        budget_report_lines.append("目前沒有預算數據可供報告。")
    else:
        for index, budget_row in budget_df.iterrows():
            budget_name = budget_row['Budget Name']
            start_date = budget_row['Start Date'].strftime('%Y-%m-%d') if pd.notna(budget_row['Start Date']) else 'N/A'
            end_date = budget_row['End Date'].strftime('%Y-%m-%d') if pd.notna(budget_row['End Date']) else 'N/A'
            budgeted_amount = budget_row['Amount']
            actual_spending = budget_row['Spending']
            remaining_overspent = budget_row['Remaining/Overspent']
            status = budget_row['Status']
            category = budget_row['Category'] if pd.notna(budget_row['Category']) and budget_row['Category'] != '' else '總體'

            budget_report_lines.append(f"\n預算名稱: {budget_name}")
            budget_report_lines.append(f"期間: {start_date} 至 {end_date}")
            budget_report_lines.append(f"類別: {category}")
            budget_report_lines.append(f"預算金額: {budgeted_amount:.2f}")
            budget_report_lines.append(f"實際支出: {actual_spending:.2f}")
            budget_report_lines.append(f"剩餘/超支金額: {remaining_overspent:.2f}")
            budget_report_lines.append(f"狀態: {status}")

    return "\n".join(budget_report_lines)

# Report generation functions
def generate_periodic_report_string(df, report_start_date, report_end_date, report_period_name):
    """Generates a formatted periodic report string based on a date range."""
    # Ensure the 'date' column in the expense DataFrame is in date format for comparison
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # Filter the main expense DataFrame (`df`) to include only transactions within the report_start_date and report_end_date.
    filtered_spending_for_report = df[(df['date'] >= report_start_date) & (df['date'] <= report_end_date)].copy()

    # Calculate the total spending for this filtered period.
    total_spending_period = filtered_spending_for_report['amount'].sum()

    # Calculate the category subtotals for the filtered spending data.
    category_subtotals_period = filtered_spending_for_report.groupby('category')['amount'].sum().reset_index()

    # Calculate the total amount paid by each person within the filtered period.
    payer_totals_period = filtered_spending_for_report.groupby('payer')['amount'].sum().reset_index()

    # Format the calculated total spending, category subtotals, and payer totals into a readable report string.
    periodic_report_lines = [f"=== {report_period_name} ({report_start_date.strftime('%Y-%m-%d')} 至 {report_end_date.strftime('%Y-%m-%d')}) 支出報告 ==="]
    periodic_report_lines.append(f"\n總支出金額: {total_spending_period:.2f}元")

    if not category_subtotals_period.empty:
        periodic_report_lines.append("\n各類別支出小計:")
        for index, row in category_subtotals_period.iterrows():
            periodic_report_lines.append(f"- {row['category']}: {row['amount']:.2f}元")
    else:
        periodic_report_lines.append("\n此期間沒有類別支出數據。")

    if not payer_totals_period.empty:
        periodic_report_lines.append("\n每人總支付金額:")
        for index, row in payer_totals_period.iterrows():
            periodic_report_lines.append(f"- {row['payer']}: {row['amount']:.2f}元")
    else:
        periodic_report_lines.append("\n此期間沒有支付者數據。")

    return "\n".join(periodic_report_lines)

def write_reports_sheet(gsheets_obj, periodic_report_string, budget_report_string):
    """Writes the periodic and budget reports to the 'Reports' sheet."""
    try:
        reports_sheet = gsheets_obj.worksheet('Reports')
        print("Sheet 'Reports' already exists, clearing it.")
        reports_sheet.clear()
    except gspread.WorksheetNotFound:
        print("Creating new sheet 'Reports'.")
        reports_sheet = gsheets_obj.add_worksheet(title="Reports", rows="200", cols="20")

    periodic_report_start_row_reports = 1
    budget_report_start_row_reports = periodic_report_start_row_reports + len(periodic_report_string.split('\n')) + 5

    reports_sheet.update_cell(periodic_report_start_row_reports, 1, '=== 週期支出報告 ===')
    periodic_report_lines = periodic_report_string.split('\n')
    if periodic_report_lines:
        periodic_report_data_to_write = [[line] for line in periodic_report_lines]
        periodic_report_range = f'A{periodic_report_start_row_reports + 1}'
        reports_sheet.update(periodic_report_data_to_write, periodic_report_range)

    reports_sheet.update_cell(budget_report_start_row_reports, 1, '=== 預算報告 ===')
    budget_report_lines = budget_report_string.split('\n')
    if budget_report_lines:
        budget_report_data_to_write = [[line] for line in budget_report_lines]
        budget_report_range = f'A{budget_report_start_row_reports + 1}'
        reports_sheet.update(budget_report_data_to_write, budget_report_range)

    print("週期支出報告和預算報告已寫入 Reports 分頁！")

def generate_spending_analysis_and_advice(df):
    """Generates spending analysis and advice using the Gemini API."""
    if gemini_model is None:
        return "AI 分析與建議功能目前無法使用，請檢查您的 API 金鑰和模型初始化狀態。"

    if df.empty:
        return "沒有足夠的支出數據來進行 AI 分析。"

    # Prepare a prompt for the AI
    # You can customize this prompt based on the insights you want from the AI
    prompt = f"""
    請分析以下支出數據，並提供總結以及關於如何改進消費習慣的建議。數據格式為 DataFrame 的 to_string() 輸出：

    {df.to_string()}

    請提供以下內容：
    1. 總體支出摘要
    2. 各類別支出分析
    3. 基於數據的個人化消費建議
    """

    try:
        # Generate content using the Gemini model
        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"AI 分析與建議生成失敗: {e}"

# Category suggestion dictionary (can be expanded)
category_suggestions = {
    '便當': '餐飲',
    '拉麵': '餐飲',
    '油錢': '交通',
    '衛生紙': '購物',
    '珍奶': '飲料',
    '星巴克': '飲料',
    '咖啡': '飲料',
    '麵包': '餐飲',
    '電影票': '娛樂',
    '公車': '交通',
    '捷運': '交通',
    '計程車': '交通',
    '衣服': '購物',
    '書本': '購物',
    '水電費': '帳單',
    '房租': '居住',
    '薪水': '收入' # Although not an expense, useful for tracking
}

# Define the Google Sheet URL
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1KekcvnaCy4MsLsbi04U0KCeys7dnh-JxCCeHkk9xKFE/edit?usp=sharing'

Gemini API configured successfully.
Gemini model initialized.


**Reasoning**:
Define the main interface logic function `main_interface_logic` and the Gradio interface `demo`, ensuring all necessary functions defined in the previous step are available before this code block is executed.



In [34]:
import gradio as gr

def main_interface_logic(action, date_str=None, category=None, item=None, amount=None, payer=None, report_period_choice=None, custom_start_date_str=None, custom_end_date_str=None):
    """
    Orchestrates the data processing and reporting based on Gradio user input.
    """
    gsheets_obj = authenticate_and_connect(SHEET_URL)
    df = read_expense_data(gsheets_obj)

    output_message = ""
    periodic_report_output = ""
    budget_report_output = ""
    aa_split_output = ""
    payer_balance_output = ""
    ai_analysis_output = "" # Added AI analysis output variable


    if action == "add_expense":
        if date_str and category and item and amount is not None and payer:
            try:
                # Validate date format
                datetime.datetime.strptime(date_str, '%Y-%m-%d')
                # Ensure amount is a float
                amount = float(amount)

                df = add_expense_record(df, date_str, category, item, amount, payer)
                write_expense_data(df, gsheets_obj)
                # Recalculate and write summary after adding expense
                total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
                # Generate AI analysis and advice
                ai_analysis_output = generate_spending_analysis_and_advice(df)
                write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output) # Pass AI analysis to write_summary_sheet
                output_message = "新的支出紀錄已成功新增並更新至 Google Sheet 及 Summary 分頁。"
                aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
                payer_balance_output = payer_balance.to_string(index=False)
            except ValueError as e:
                output_message = f"輸入錯誤: {e}. 請檢查日期和金額格式。"
        else:
            output_message = "請填寫所有支出紀錄欄位。"
        return output_message, "", "", aa_split_output, payer_balance_output, ai_analysis_output # Include AI analysis output

    elif action == "generate_reports":
        if report_period_choice:
            today = datetime.date.today()
            report_start_date = None
            report_end_date = None
            report_period_name = ""

            if report_period_choice == '上週':
                last_sunday = today - relativedelta(days=today.weekday() + 1)
                last_monday = last_sunday - relativedelta(days=6)
                report_start_date = last_monday
                report_end_date = last_sunday
                report_period_name = "上週"
            elif report_period_choice == '上個月':
                first_day_of_current_month = today.replace(day=1)
                last_day_of_last_month = first_day_of_current_month - relativedelta(days=1)
                first_day_of_last_month = last_day_of_last_month.replace(day=1)
                report_start_date = first_day_of_last_month
                report_end_date = last_day_of_last_month
                report_period_name = "上個月"
            elif report_period_choice == '自訂日期範圍':
                if custom_start_date_str and custom_end_date_str:
                    try:
                        report_start_date = datetime.datetime.strptime(custom_start_date_str, '%Y-%m-%d').date()
                        report_end_date = datetime.datetime.strptime(custom_end_date_str, '%Y-%m-%d').date()
                        if report_start_date > report_end_date:
                            output_message = "自訂日期範圍錯誤：開始日期不能晚於結束日期。"
                            return output_message, "", "", "", "", "" # Include "" for all outputs
                        report_period_name = "自訂日期範圍"
                    except ValueError:
                        output_message = "自訂日期格式錯誤，請使用 YYYY-MM-DD 格式。"
                        return output_message, "", "", "", "", "" # Include "" for all outputs
                else:
                    output_message = "請輸入自訂報告的開始日期和結束日期。"
                    return output_message, "", "", "", "", "" # Include "" for all outputs

            # Generate Periodic Report
            if report_start_date and report_end_date:
                 periodic_report_output = generate_periodic_report_string(df, report_start_date, report_end_date, report_period_name)
            else:
                 periodic_report_output = "請選擇或輸入有效的報告期間。"

            # Generate Budget Report
            budget_df = get_budget_data(gsheets_obj)
            budget_df = calculate_spending_within_budgets(df, budget_df)
            budget_df = compare_spending_to_budget(budget_df)
            budget_report_output = generate_budget_report_string(budget_df)

            # Write reports to Reports sheet
            write_reports_sheet(gsheets_obj, periodic_report_output, budget_report_output)
            output_message = "報告已生成並更新至 Reports 分頁。"
        else:
            output_message = "請選擇報告期間。"
        return output_message, periodic_report_output, budget_report_output, "", "", "" # Include "" for AA and AI analysis outputs

    elif action == "calculate_aa":
        total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
        aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
        payer_balance_output = payer_balance.to_string(index=False)
        output_message = "AA 分攤金額和每人應收/應付金額已計算。"
        return output_message, "", "", aa_split_output, payer_balance_output, "" # Include "" for AI analysis output

    elif action == "generate_ai_analysis": # Added action for generating AI analysis on demand
         ai_analysis_output = generate_spending_analysis_and_advice(df)
         output_message = "AI 支出分析與建議已生成。"
         # Update the Summary sheet with the new AI analysis
         total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
         write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output)
         return output_message, "", "", "", "", ai_analysis_output # Return only AI analysis output and status

# Define Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 日常支出管理與報告")

    with gr.Tab("新增支出紀錄"):
        gr.Markdown("在此輸入新的支出紀錄。")
        expense_date = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder="例如：2023-10-27")
        expense_category = gr.Textbox(label="類別", placeholder="例如：餐飲、交通")
        expense_item = gr.Textbox(label="品項", placeholder="例如：午餐、公車票")
        expense_amount = gr.Number(label="金額", placeholder="例如：100")
        expense_payer = gr.Textbox(label="支付者", placeholder="例如：我, 朋友A")
        add_expense_btn = gr.Button("新增紀錄")
        add_expense_output = gr.Textbox(label="狀態訊息")

        add_expense_btn.click(
            fn=lambda d, c, i, a, p: main_interface_logic("add_expense", date_str=d, category=c, item=i, amount=a, payer=p),
            inputs=[expense_date, expense_category, expense_item, expense_amount, expense_payer],
            outputs=[add_expense_output, gr.State(), gr.State(), gr.State(), gr.State(), gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("生成報告"):
        gr.Markdown("選擇期間以生成支出報告和預算報告。")
        report_period_selection = gr.Dropdown(
            choices=['上週', '上個月', '自訂日期範圍'],
            label="選擇報告期間"
        )
        custom_start_date = gr.Textbox(label="自訂開始日期 (YYYY-MM-DD)", placeholder="例如：2023-10-01", visible=False)
        custom_end_date = gr.Textbox(label="自訂結束日期 (YYYY-MM-DD)", placeholder="例如：2023-10-31", visible=False)

        # Show/hide custom date fields based on dropdown selection
        def toggle_custom_dates(choice):
            if choice == '自訂日期範圍':
                return gr.update(visible=True), gr.update(visible=True)
            else:
                return gr.update(visible=False), gr.update(visible=False)

        report_period_selection.change(
            fn=toggle_custom_dates,
            inputs=report_period_selection,
            outputs=[custom_start_date, custom_end_date]
        )

        generate_report_btn = gr.Button("生成報告")
        report_status_output = gr.Textbox(label="報告生成狀態")
        periodic_report_display = gr.Textbox(label="週期支出報告", lines=10)
        budget_report_display = gr.Textbox(label="預算報告", lines=10)

        generate_report_btn.click(
            fn=lambda choice, start_date_str, end_date_str: main_interface_logic("generate_reports", report_period_choice=choice, custom_start_date_str=start_date_str, custom_end_date_str=end_date_str),
            inputs=[report_period_selection, custom_start_date, custom_end_date],
            outputs=[report_status_output, periodic_report_display, budget_report_display, gr.State(), gr.State(), gr.State()] # Added gr.State() for AA and AI analysis outputs
        )

    with gr.Tab("分帳與計算 (AA)"): # New tab for AA Split
        gr.Markdown("在此計算 AA 分攤金額和每人應收/應付金額。")
        calculate_aa_btn = gr.Button("計算 AA 分攤")
        aa_calculation_status = gr.Textbox(label="狀態訊息")
        aa_split_display = gr.Textbox(label="AA 分攤金額", lines=1)
        payer_balance_display = gr.Textbox(label="每人應收/應付金額", lines=5)

        calculate_aa_btn.click(
            fn=lambda: main_interface_logic("calculate_aa"),
            inputs=[],
            outputs=[aa_calculation_status, gr.State(), gr.State(), aa_split_display, payer_balance_display, gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("AI 支出分析"): # New tab for AI Analysis
        gr.Markdown("在此獲取 AI 支出的分析與建議。")
        generate_ai_analysis_btn = gr.Button("生成 AI 分析")
        ai_analysis_status = gr.Textbox(label="狀態訊息")
        ai_analysis_display = gr.Textbox(label="AI 分析與建議", lines=15)

        generate_ai_analysis_btn.click(
            fn=lambda: main_interface_logic("generate_ai_analysis"),
            inputs=[],
            outputs=[ai_analysis_status, gr.State(), gr.State(), gr.State(), gr.State(), ai_analysis_display] # Output AI analysis display
        )

**Reasoning**:
Launch the Gradio application to allow user interaction and testing of the implemented functionalities.



In [29]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41fd371935af86bcd9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 移除無用儲存格

### Subtask:
刪除不再需要的儲存格，例如只有匯入語句、過時的測試程式碼或重複的函式定義。


## 測試精簡後的程式碼

### Subtask:
執行精簡後的程式碼，確認所有功能（新增支出、生成報告、AA 分攤）都能正常運作，並且不再出現錯誤。


**Reasoning**:
Execute the code cell containing the Gradio interface definition to create the user interface.



In [30]:
import gradio as gr

def main_interface_logic(action, date_str=None, category=None, item=None, amount=None, payer=None, report_period_choice=None, custom_start_date_str=None, custom_end_date_str=None):
    """
    Orchestrates the data processing and reporting based on Gradio user input.
    """
    gsheets_obj = authenticate_and_connect(SHEET_URL)
    df = read_expense_data(gsheets_obj)

    output_message = ""
    periodic_report_output = ""
    budget_report_output = ""
    aa_split_output = ""
    payer_balance_output = ""
    ai_analysis_output = "" # Added AI analysis output variable


    if action == "add_expense":
        if date_str and category and item and amount is not None and payer:
            try:
                # Validate date format
                datetime.datetime.strptime(date_str, '%Y-%m-%d')
                # Ensure amount is a float
                amount = float(amount)

                df = add_expense_record(df, date_str, category, item, amount, payer)
                write_expense_data(df, gsheets_obj)
                # Recalculate and write summary after adding expense
                total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
                # Generate AI analysis and advice
                ai_analysis_output = generate_spending_analysis_and_advice(df)
                write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output) # Pass AI analysis to write_summary_sheet
                output_message = "新的支出紀錄已成功新增並更新至 Google Sheet 及 Summary 分頁。"
                aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
                payer_balance_output = payer_balance.to_string(index=False)
            except ValueError as e:
                output_message = f"輸入錯誤: {e}. 請檢查日期和金額格式。"
        else:
            output_message = "請填寫所有支出紀錄欄位。"
        return output_message, "", "", aa_split_output, payer_balance_output, ai_analysis_output # Include AI analysis output

    elif action == "generate_reports":
        if report_period_choice:
            today = datetime.date.today()
            report_start_date = None
            report_end_date = None
            report_period_name = ""

            if report_period_choice == '上週':
                last_sunday = today - relativedelta(days=today.weekday() + 1)
                last_monday = last_sunday - relativedelta(days=6)
                report_start_date = last_monday
                report_end_date = last_sunday
                report_period_name = "上週"
            elif report_period_choice == '上個月':
                first_day_of_current_month = today.replace(day=1)
                last_day_of_last_month = first_day_of_current_month - relativedelta(days=1)
                first_day_of_last_month = last_day_of_last_month.replace(day=1)
                report_start_date = first_day_of_last_month
                report_end_date = last_day_of_last_month
                report_period_name = "上個月"
            elif report_period_choice == '自訂日期範圍':
                if custom_start_date_str and custom_end_date_str:
                    try:
                        report_start_date = datetime.datetime.strptime(custom_start_date_str, '%Y-%m-%d').date()
                        report_end_date = datetime.datetime.strptime(custom_end_date_str, '%Y-%m-%d').date()
                        if report_start_date > report_end_date:
                            output_message = "自訂日期範圍錯誤：開始日期不能晚於結束日期。"
                            return output_message, "", "", "", "", "" # Include "" for all outputs
                        report_period_name = "自訂日期範圍"
                    except ValueError:
                        output_message = "自訂日期格式錯誤，請使用 YYYY-MM-DD 格式。"
                        return output_message, "", "", "", "", "" # Include "" for all outputs
                else:
                    output_message = "請輸入自訂報告的開始日期和結束日期。"
                    return output_message, "", "", "", "", "" # Include "" for all outputs

            # Generate Periodic Report
            if report_start_date and report_end_date:
                 periodic_report_output = generate_periodic_report_string(df, report_start_date, report_end_date, report_period_name)
            else:
                 periodic_report_output = "請選擇或輸入有效的報告期間。"

            # Generate Budget Report
            budget_df = get_budget_data(gsheets_obj)
            budget_df = calculate_spending_within_budgets(df, budget_df)
            budget_df = compare_spending_to_budget(budget_df)
            budget_report_output = generate_budget_report_string(budget_df)

            # Write reports to Reports sheet
            write_reports_sheet(gsheets_obj, periodic_report_output, budget_report_output)
            output_message = "報告已生成並更新至 Reports 分頁。"
        else:
            output_message = "請選擇報告期間。"
        return output_message, periodic_report_output, budget_report_output, "", "", "" # Include "" for AA and AI analysis outputs

    elif action == "calculate_aa":
        total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
        aa_split_output = f"AA 分攤金額 (每人): {aa_split_amount:.2f}元"
        payer_balance_output = payer_balance.to_string(index=False)
        output_message = "AA 分攤金額和每人應收/應付金額已計算。"
        return output_message, "", "", aa_split_output, payer_balance_output, "" # Include "" for AI analysis output

    elif action == "generate_ai_analysis": # Added action for generating AI analysis on demand
         ai_analysis_output = generate_spending_analysis_and_advice(df)
         output_message = "AI 支出分析與建議已生成。"
         # Update the Summary sheet with the new AI analysis
         total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance = calculate_summary_statistics(df)
         write_summary_sheet(gsheets_obj, total_amount, category_subtotals, aa_split_amount, payer_totals, payer_balance, spending_analysis_and_advice=ai_analysis_output)
         return output_message, "", "", "", "", ai_analysis_output # Return only AI analysis output and status

# Define Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 日常支出管理與報告")

    with gr.Tab("新增支出紀錄"):
        gr.Markdown("在此輸入新的支出紀錄。")
        expense_date = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder="例如：2023-10-27")
        expense_category = gr.Textbox(label="類別", placeholder="例如：餐飲、交通")
        expense_item = gr.Textbox(label="品項", placeholder="例如：午餐、公車票")
        expense_amount = gr.Number(label="金額", placeholder="例如：100")
        expense_payer = gr.Textbox(label="支付者", placeholder="例如：我, 朋友A")
        add_expense_btn = gr.Button("新增紀錄")
        add_expense_output = gr.Textbox(label="狀態訊息")

        add_expense_btn.click(
            fn=lambda d, c, i, a, p: main_interface_logic("add_expense", date_str=d, category=c, item=i, amount=a, payer=p),
            inputs=[expense_date, expense_category, expense_item, expense_amount, expense_payer],
            outputs=[add_expense_output, gr.State(), gr.State(), gr.State(), gr.State(), gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("生成報告"):
        gr.Markdown("選擇期間以生成支出報告和預算報告。")
        report_period_selection = gr.Dropdown(
            choices=['上週', '上個月', '自訂日期範圍'],
            label="選擇報告期間"
        )
        custom_start_date = gr.Textbox(label="自訂開始日期 (YYYY-MM-DD)", placeholder="例如：2023-10-01", visible=False)
        custom_end_date = gr.Textbox(label="自訂結束日期 (YYYY-MM-DD)", placeholder="例如：2023-10-31", visible=False)

        # Show/hide custom date fields based on dropdown selection
        def toggle_custom_dates(choice):
            if choice == '自訂日期範圍':
                return gr.update(visible=True), gr.update(visible=True)
            else:
                return gr.update(visible=False), gr.update(visible=False)

        report_period_selection.change(
            fn=toggle_custom_dates,
            inputs=report_period_selection,
            outputs=[custom_start_date, custom_end_date]
        )

        generate_report_btn = gr.Button("生成報告")
        report_status_output = gr.Textbox(label="報告生成狀態")
        periodic_report_display = gr.Textbox(label="週期支出報告", lines=10)
        budget_report_display = gr.Textbox(label="預算報告", lines=10)

        generate_report_btn.click(
            fn=lambda choice, start_date_str, end_date_str: main_interface_logic("generate_reports", report_period_choice=choice, custom_start_date_str=start_date_str, custom_end_date_str=end_date_str),
            inputs=[report_period_selection, custom_start_date, custom_end_date],
            outputs=[report_status_output, periodic_report_display, budget_report_display, gr.State(), gr.State(), gr.State()] # Added gr.State() for AA and AI analysis outputs
        )

    with gr.Tab("分帳與計算 (AA)"): # New tab for AA Split
        gr.Markdown("在此計算 AA 分攤金額和每人應收/應付金額。")
        calculate_aa_btn = gr.Button("計算 AA 分攤")
        aa_calculation_status = gr.Textbox(label="狀態訊息")
        aa_split_display = gr.Textbox(label="AA 分攤金額", lines=1)
        payer_balance_display = gr.Textbox(label="每人應收/應付金額", lines=5)

        calculate_aa_btn.click(
            fn=lambda: main_interface_logic("calculate_aa"),
            inputs=[],
            outputs=[aa_calculation_status, gr.State(), gr.State(), aa_split_display, payer_balance_display, gr.State()] # Added gr.State() for AI analysis output
        )

    with gr.Tab("AI 支出分析"): # New tab for AI Analysis
        gr.Markdown("在此獲取 AI 支出的分析與建議。")
        generate_ai_analysis_btn = gr.Button("生成 AI 分析")
        ai_analysis_status = gr.Textbox(label="狀態訊息")
        ai_analysis_display = gr.Textbox(label="AI 分析與建議", lines=15)

        generate_ai_analysis_btn.click(
            fn=lambda: main_interface_logic("generate_ai_analysis"),
            inputs=[],
            outputs=[ai_analysis_status, gr.State(), gr.State(), gr.State(), gr.State(), ai_analysis_display] # Output AI analysis display
        )

### Subtask:
Examine the `main_interface_logic` function and the Gradio interface definition to ensure the AI analysis functionality is correctly implemented and connected.

**Reasoning**:
Launch the Gradio application to allow user interaction and testing of the implemented functionalities.



In [31]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba6505b1c33f0aa166.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 啟動 gradio 介面

### Subtask:
啟動更新後的 Gradio 介面，提供一個可分享的連結讓使用者測試。


**Reasoning**:
Launch the Gradio application with the defined interface (`demo`) and generate a shareable link.



In [35]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://18fece2f87ca34c4c7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   The initial analysis of the notebook revealed significant code redundancy, particularly in function definitions for Google Sheets interaction, summary calculations, and reporting, which were defined multiple times across different cells.
*   Potential `NameError` issues related to the Gradio `demo` object launch order and `TypeError` with `int64` serialization for Google Sheets were identified during the analysis and subsequently addressed in the consolidated code.
*   All necessary imports and function definitions for Google Sheets interaction, summary statistics, budget handling, report generation, and AI analysis were successfully consolidated into a single code cell.
*   The `main_interface_logic` function was redefined to orchestrate different actions ("add\_expense", "generate\_reports", "calculate\_aa", "generate\_ai\_analysis") based on user input from the Gradio interface.
*   The Gradio interface was successfully redefined with four distinct tabs for "新增支出紀錄", "生成報告", "分帳與計算 (AA)", and "AI 支出分析", with inputs and outputs correctly linked to the `main_interface_logic`.
*   The `TypeError: Object of type int64 is not JSON serializable` was resolved by converting numeric types to `float` or `str` before writing to Google Sheets in the `write_summary_sheet` and `write_expense_data` functions.
*   The Gradio application was successfully launched and a public shareable URL was generated, although a `GOOGLE_API_KEY` configuration issue for the Gemini API was noted, which might affect the AI analysis feature.

### Insights or Next Steps

*   Verify the functionality of each tab in the launched Gradio application, including adding new expenses, generating reports (periodic and budget), calculating AA splits, and generating AI analysis, by interacting with the web interface using the provided shareable link.
*   Address the `GOOGLE_API_KEY` configuration issue to ensure the AI analysis feature functions correctly. This may involve adding the API key to Colab secrets or providing a valid key through another secure method.
